In [50]:
import re, pandas as pd, numpy as np, os
from io import StringIO
from IPython.core.display import display
import time # time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(1634309423832/1000))
# print(pd.api.types.infer_dtype([78.064002990723,78.064002990723]))
destfolder = 'C:/Beachhead/Misc/LogTinker/outdata/'
csvdtype = {'seq': int, 'timestamp': 'int64', 'player': 'category', 'event': 'category'}
csvNumArgs = ["arg"+str(i) for i in range(1, 25)]
for arg in csvNumArgs: csvdtype[arg] = 'str'
csvcols = ";".join(csvdtype.keys())
jsondat = pd.read_json(StringIO('''{"events":[
   {"name":"EVENT_COMBAT_EVENT", "arguments":[
      {"name":"result", "type":"category"},
      {"name":"isError", "type":"category"},
      {"name":"abilityName", "type":"category"},
      {"name":"abilityGraphic", "type":"category"},
      {"name":"abilityActionSlotType", "type":"category"},
      {"name":"sourceName", "type":"category"},
      {"name":"sourceType", "type":"category"},
      {"name":"targetName", "type":"category"},
      {"name":"targetType", "type":"category"},
      {"name":"hitValue", "type":"int"},
      {"name":"powerType", "type":"category"},
      {"name":"damageType", "type":"category"},
      {"name":"log", "type":"category"},
      {"name":"sourceUnitId", "type":"category"},
      {"name":"targetUnitId", "type":"category"},
      {"name":"abilityId", "type":"category"},
      {"name":"overflow", "type":"category"}
   ]},
   {"name":"EVENT_EFFECT_CHANGED", "arguments":[
      {"name":"changeType", "type":"category"},
      {"name":"effectSlot", "type":"int"},
      {"name":"effectName", "type":"category"},
      {"name":"unitTag", "type":"category"},
      {"name":"beginTime", "type":"float"},
      {"name":"endTime", "type":"float"},
      {"name":"stackCount", "type":"category"},
      {"name":"iconName", "type":"category"},
      {"name":"buffType", "type":"category"},
      {"name":"effectType", "type":"category"},
      {"name":"abilityType", "type":"category"},
      {"name":"statusEffectType", "type":"category"},
      {"name":"unitName", "type":"category"},
      {"name":"unitId", "type":"int"},
      {"name":"abilityId", "type":"int"},
      {"name":"sourceType", "type":"category"}
   ]}
]}'''))
if not os.path.exists(destfolder+'ALL'): os.makedirs(destfolder+'ALL')
events = []
for jsonEvent in jsondat["events"]:
   eventName = jsonEvent['name']
   destEventPath = destfolder+eventName
   if not os.path.exists(destEventPath): os.makedirs(destEventPath)
   eventArguments = jsonEvent['arguments']
   eventDfCols = ['seq', 'timestamp', 'player']
   eventDfCols.extend(csvNumArgs[:len(eventArguments)])
   eventArgRename = dict(zip(csvNumArgs[:len(eventArguments)], [x['name'] for x in eventArguments]))
   eventArgTypes = {x['name']:x['type'] for x in eventArguments}
   events.append({'eventName':eventName, 'destEventPath':destEventPath, 'eventDfCols':eventDfCols, 'eventArgRename':eventArgRename, 'eventArgTypes':eventArgTypes})

def loadRaw(sourcefilename):
    with open(sourcefilename, "r", encoding='utf8') as sourcefile:
        pseudocsv = sourcefile.read()
        pseudocsv = pseudocsv[pseudocsv.find("[1]"):pseudocsv.rfind("\",")+2]
        pseudocsv = pseudocsv.replace("\\\"",":quot:")
        pseudocsv = re.sub("\[(\d+)\]\s?=\s?\"(.*?)\",\s*", "\\1;\\2\\n", pseudocsv)
        pseudocsv = pseudocsv.replace(":quot:","\\\"")
        return pd.read_csv(StringIO(csvcols+"\n"+pseudocsv), dtype=csvdtype, sep=";", header=0)
def mergeDataFrameIntoFile(eventDataFrame, fileNameEventDate): # pd.read_parquet(fileNameEventDate).append(eventDataFrame).drop_duplicates(subset=["seq", "timestamp", "player"]).to_parquet(fileNameEventDate)
   try:
      fileDataFrame = pd.read_parquet(fileNameEventDate)
   except FileNotFoundError:
      eventDataFrame.to_parquet(fileNameEventDate) # create new file since none exists
   else:
      cnt = len(fileDataFrame)
      fileDataFrame = fileDataFrame.append(eventDataFrame)
      fileDataFrame.drop_duplicates(subset=["seq", "timestamp", "player"], inplace=True)
      if len(fileDataFrame) != cnt :
         fileDataFrame.to_parquet(fileNameEventDate) # appending records made a difference so update the file

rawfiles = ["B916DF15A205777A0E6C358803A6E204.lson", "2136039DCAB1882ACD05B5FAE568858D.lson", "E0B499D2E7E6CBA2868D3830BE4EDE4C.lson", "E376DE68C859549B2CD3F9629B3DD5DB.lson"]
for rawfile in rawfiles:
   df = loadRaw(rawfile)
   fileprefix = time.strftime('%Y-%m-%d', time.localtime(df['timestamp'].values[-1]/1000))
   fileNameEventDate = destfolder+'ALL/'+fileprefix+'.parquet'
   print(fileNameEventDate)
   # df.to_parquet(destfolder+'ALL/'+fileprefix+'.parquet')
   mergeDataFrameIntoFile(df, fileNameEventDate)
   for e in events:
      eventDataFrame = pd.DataFrame(df[df["event"] == e['eventName']], columns=e['eventDfCols']).rename(columns=e['eventArgRename']).astype(dtype=e['eventArgTypes'])
      fileNameEventDate = e['destEventPath']+'/'+fileprefix+'.parquet'
      mergeDataFrameIntoFile(eventDataFrame, fileNameEventDate)



C:/Beachhead/Misc/LogTinker/outdata/ALL/2021-09-25.parquet
C:/Beachhead/Misc/LogTinker/outdata/ALL/2021-09-20.parquet
C:/Beachhead/Misc/LogTinker/outdata/ALL/2021-09-20.parquet
C:/Beachhead/Misc/LogTinker/outdata/ALL/2021-10-01.parquet
